In [12]:
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
import yfinance as yf
from pathlib import Path
import altair as alt
import plotly.graph_objects as go

In [13]:
def get_top_10_stock_list():
    marketcap_csv_path = Path("Resources/marketcap.csv")
    marketcap_df = pd.read_csv(marketcap_csv_path)
    ticker_marketcap_df = marketcap_df.loc[:, ['Year','Ticker', 'MarketCap(USD Millions)']]
    #ticker_marketcap_df.sort_values("Ticker")

    ticker_marketcap_df['MarketCap(USD Millions)'].replace(',','', regex=True, inplace=True)
    ticker_marketcap_df['MarketCap(USD Millions)'] = ticker_marketcap_df['MarketCap(USD Millions)'].astype(int)
    #ticker_marketcap_df['MarketCap(USD Millions)'] = ticker_marketcap_df['MarketCap(USD Millions)'].apply(pd.to_numeric)
    ticker_marketcap_df = ticker_marketcap_df[ticker_marketcap_df['MarketCap(USD Millions)'] != 0]
    return ticker_marketcap_df


def get_raw_data():
    stock_data_csv_path = Path("Resources/stock_data.csv")
    stock_data_df = pd.read_csv(stock_data_csv_path)
    stock_data_df['Date'] = pd.to_datetime(stock_data_df['Date'])
    stock_data_df = stock_data_df.set_index('Date')
    return stock_data_df


# Function to return the inversion plot for the gven dataframe
def get_pct_chg_plot(df, plot_name):
    df = df.pct_change().dropna()
    df = (1 + df).cumprod() - 1
    plot = px.line(df, title=plot_name)
    return plot

# Function to return all the inversion related plots in a panel
def org_inv_plots(inv_plots):
    # 10YR/2YR TSY Inversion (11/15/2006) (row #1)
    tenyr_twoyr_first_plots = pn.Row()
    for i in range(0, 3):
        tenyr_twoyr_first_plots.append(inv_plots[i])
    
    # 10YR/2YR TSY Inversion (08/29/2019) (row #2)
    tenyr_twoyr_second_plots = pn.Row()
    for i in range(3, 6):
        tenyr_twoyr_second_plots.append(inv_plots[i])
    
    # 10YR/3m TSY Inversion (02/27/2007) (row #3)
    tenyr_threemonth_first_plots = pn.Row()
    for i in range(6, 9):
        tenyr_threemonth_first_plots.append(inv_plots[i])
        
    # 10YR/3m TSY Inversion (02/25/2020) (row #4)
    tenyr_threemonth_second_plots = pn.Row()
    for i in range(9, 12):
        tenyr_threemonth_second_plots.append(inv_plots[i])

    inv_plots = pn.Column('TSY inversion plots', 
                           tenyr_twoyr_first_plots,
                           tenyr_twoyr_second_plots,
                           tenyr_threemonth_first_plots,
                           tenyr_threemonth_second_plots)
        
    return inv_plots


In [36]:
# plot of union of top performing stocks over past 15 years
def history_plot():
    raw_df = get_raw_data()
    plot = get_pct_chg_plot(raw_df, "Historical plot")
    return plot

# top 10 market cap per year
def top_market_cap_per_year():
    ticker_marketcap_df = get_top_10_stock_list()
    
    year_list = list(ticker_marketcap_df['Year'].unique())
    
    input_dropdown = alt.binding_select(options=year_list)
    selection = alt.selection_single(fields=['Year'], bind=input_dropdown, name='Year')
    alt_plot = alt.Chart(ticker_marketcap_df).mark_line().encode(
        x='Ticker',
        y='MarketCap(USD Millions)',
        tooltip='MarketCap(USD Millions)'
    ).add_selection(
        selection
    ).transform_filter(
        selection
    )
    
    fig = go.Figure()
    for year in year_list:
        fig.add_trace(
            go.Bar(
                x = ticker_marketcap_df['Ticker'][ticker_marketcap_df['Year']==year],
                y = ticker_marketcap_df['MarketCap(USD Millions)'][ticker_marketcap_df['Year']==year],
                name = str(year), visible = True,
                offsetgroup=1,
                text = ticker_marketcap_df['MarketCap(USD Millions)']/1000000,
                orientation='v'
                )
            )

    buttons = []

    for i, year in enumerate(year_list):
        args = [False] * len(year_list)
        args[i] = True

        button = dict(label = str(year),
                      method = "update",
                      args=[{"visible": args}])
        buttons.append(button)

    fig.update_layout(
        title="Choose Year",
        updatemenus=[dict(
                        active=0,
                        type="dropdown",
                        buttons=buttons,
                        x = 0,
                        y = 1.1,
                        xanchor = 'left',
                        yanchor = 'bottom'
                        )
                    ],
    autosize=False,
    width=1000,
    height=500,
    yaxis_title="Market Cap",
    xaxis_title="Stocks Leaders",
    )

    fig.update_traces(texttemplate='%{text:.2f}T', textposition='inside')
    
    return fig

    

In [22]:
def get_yield_curve_df():    

    stock_data_raw = get_raw_data()
    
    sector_data_raw = stock_data_raw.drop(columns = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'BRK-A', 'NVDA', 'FB', 'V',
       'JPM', 'JNJ', 'XOM', 'GE', 'WFC', 'WMT', 'CVX', 'IBM', 'PG', 'T', 'BAC'])
    sector_data_raw.columns = ['S&P 500','Materials','Communication Services','Energy','Financial','Industrial','Technology','Consumer Staples','Real Estate','Utilities','Healthcare','Consumer Discretionary']
    sector_data_raw
    tenyr_twoyr_first_inv_6mo = sector_data_raw.loc['2006-11-15':'2007-05-15']
    tenyr_twoyr_first_inv_12mo = sector_data_raw.loc['2006-11-15':'2007-11-15']
    tenyr_twoyr_first_inv_18mo = sector_data_raw.loc['2006-11-15':'2008-05-15']
    tenyr_twoyr_second_inv_6mo = sector_data_raw.loc['2019-08-29':'2020-02-29']
    tenyr_twoyr_second_inv_12mo = sector_data_raw.loc['2019-08-29':'2020-08-29']
    tenyr_twoyr_second_inv_18mo = sector_data_raw.loc['2019-08-29':'2021-03-01']
    tenyr_threemonth_first_inv_6mo = sector_data_raw.loc['2007-02-27':'2007-08-27']
    tenyr_threemonth_first_inv_12mo = sector_data_raw.loc['2007-02-27':'2008-02-27']
    tenyr_threemonth_first_inv_18mo = sector_data_raw.loc['2007-02-27':'2008-08-27']
    tenyr_threemonth_second_inv_6mo = sector_data_raw.loc['2020-02-25':'2020-08-25']
    tenyr_threemonth_second_inv_12mo = sector_data_raw.loc['2020-02-25':'2021-02-25']
    tenyr_threemonth_second_inv_18mo = sector_data_raw.loc['2020-02-25':'2021-08-25']
    
    #initialize all plots
    all_plots = []
    
    # 10YR/2YR TSY Inversion (11/15/2006) for 6m, 12m, 18m
    plot = get_pct_chg_plot(tenyr_twoyr_first_inv_6mo, '10YR/2YR TSY Inversion 11/15/2006 6-Month Returns')
    all_plots.append(plot)
    
    plot = get_pct_chg_plot(tenyr_twoyr_first_inv_12mo, '10YR/2YR TSY Inversion 11/15/2006 12-Month Returns')
    all_plots.append(plot)
    
    plot = get_pct_chg_plot(tenyr_twoyr_first_inv_18mo, '10YR/2YR TSY Inversion 11/15/2006 18-Month Returns')
    all_plots.append(plot)
    

    # 10YR/2YR TSY Inversion (08/29/2019) for 6m, 12m, 18m
    plot = get_pct_chg_plot(tenyr_twoyr_second_inv_6mo, '10YR/2YR TSY Inversion 08/29/2019 6-Month Returns')
    all_plots.append(plot)

    plot = get_pct_chg_plot(tenyr_twoyr_second_inv_12mo, '10YR/2YR TSY Inversion 08/29/2019 12-Month Returns')
    all_plots.append(plot)
    
    plot = get_pct_chg_plot(tenyr_twoyr_second_inv_18mo, '10YR/2YR TSY Inversion 08/29/2019 18-Month Returns')
    all_plots.append(plot)
    
    # 10YR/3M TSY Inversion (02/27/2007) for 6m, 12m, 18m
    plot = get_pct_chg_plot(tenyr_threemonth_first_inv_6mo, '10YR/3M TSY Inversion 02/27/2007 6-Month Returns')
    all_plots.append(plot)
    
    plot = get_pct_chg_plot(tenyr_threemonth_first_inv_12mo, '10YR/3M TSY Inversion 02/27/2007 12-Month Returns')
    all_plots.append(plot)
    
    plot = get_pct_chg_plot(tenyr_threemonth_first_inv_18mo, '10YR/3M TSY Inversion 02/27/2007 18-Month Returns')
    all_plots.append(plot)

    # 10YR/3M TSY Inversion (02/25/2020) for 6m, 12m, 18m
    plot = get_pct_chg_plot(tenyr_threemonth_second_inv_6mo, '10YR/3M TSY Inversion 02/25/2020 6-Month Returns')
    all_plots.append(plot)
    
    plot = get_pct_chg_plot(tenyr_threemonth_second_inv_12mo, '10YR/3M TSY Inversion 02/25/2020 12-Month Returns')
    all_plots.append(plot)
    
    plot = get_pct_chg_plot(tenyr_threemonth_second_inv_18mo, '10YR/3M TSY Inversion 02/25/2020 18-Month Returns')
    all_plots.append(plot)
    
    return all_plots

In [37]:
# historical plots

history_plots = pn.Column("Historical data",
                             history_plot())

# Market cap related plots
market_cap_plots = pn.Column("Market cap yearly",
                             top_market_cap_per_year())

# Performance related plots
#perf_plots = pn.Column("Performance yearly",
#                        top_perf_per_year())

#fetch all the yield curve related plots in a list
yc_plots = org_inv_plots(get_yield_curve_df())

# Create the dashboard
tabs = pn.Tabs(
              ("Market cap per year", market_cap_plots),
              ("Yield curve window performance", yc_plots),
              )

tabs

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
            [2] Plotly(Figure)
        [2] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
            [2] Plotly(Figure)
        [3] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
            [2] Plotly(Figure)
        [4] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)
            [2] Plotly(Figure)

In [ ]:
#get_raw_data().head()